<a href="https://colab.research.google.com/github/luke-scot/s4-thermal/blob/main/buildingProperties.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Buidling properties from OSM data

This script takes geojson files from [Overpass for OSM](https://overpass-turbo.eu/#) and outputs a clickable .html map and a .csv data file which include the roof and facade areas of each building. Defaults are set for Cambridge University, but you can change the application to any location by changing the variables.

To obtain a new input geojson, go to [Overpass](https://overpass-turbo.eu/#) and draw a bbox using the selection in the little list on the left. Then press run at the top right of the screen. Finally Export as a geojson. Then simply upload the geoJson to this Colab environment using the upload button in the left bar. 

Any problems let me know at lshc3@cam.ac.uk. Enjoy!

In [ ]:
# Import packages
%%capture
%pip install geopandas gdown
import geopandas as gpd
import pandas as pd
import folium
import numpy as np
import os

In [ ]:
# Define variables
# Use geojson with all Cambridge building footprints (only use when necessary as
# is large - otherwise go to Overpass and define your own geojson)
allCambridge = True 
buildings = 'camBuildings.geojson' # If own geojson
crs = 'EPSG:32631' # Coordinate system
camUniRef = True # If using cambridge uni reference file
refFile = 'camUniBuildings.csv' # Name of reference file if other than cam uni
refsOnly = True # True means only outputs data for buildings contained in reference file
floorHeight = 3  # Standard metres per floor
unknownFloors = 2 # When floor number is unknown use this
name = 'CambridgeUni' # Ouput file name 

In [ ]:
# Downloading default files if required
if camUniRef: 
  ! gdown -O $refFile "https://drive.google.com/uc?export=download&id=1eM8K5MO3FYJDAFz_9Po8UDKg9xD-uvH9"
if allCambridge:
  ! gdown -O $buildings "https://drive.google.com/uc?export=download&id=1gpEKjm6oSkBb6X-QdjYOskw9d6gMAPRS"

# Merging data from files into dataframe
if os.path.isfile(refFile): # If reference file exists
  refDf = pd.read_csv(refFile)
  buildsGdf = gpd.read_file(buildings).to_crs(crs)
  if refsOnly: gdf = pd.merge(buildsGdf, refDf, left_on='ref', right_on='Building Code')
  else: gdf = pd.merge(buildsGdf, refDf, how='left', left_on='ref', right_on='Building Code')
else: gdf = gpd.read_file(buildings).to_crs(crs) # If there is no reference file

# This will show the first 5 rows of the dataframe
gdf.head()

In [ ]:
# Calculating floor, area and facade values
gdf['area'] = gdf.geometry.area
if os.path.isfile(refFile): gdf['floors'] = [gdf.loc[i]['No. Floors'] if 'NaN' not in str(gdf.loc[i]['No. Floors']) else gdf.loc[i]['building:levels'] for i in range(len(gdf))]
else: gdf['floors'] = gdf.loc[i]['building:levels']
gdf['floors'].replace(to_replace=np.NaN, value=unknownFloors, inplace=True)
gdf['facade'] = gdf['floors'].astype(int)*floorHeight*gdf.geometry.length
gll = gdf[['Point' not in str(type(gdf.loc[i].geometry)) for i in range(len(gdf))]].reindex().to_crs('epsg:4326')

# Plotting map
m = folium.Map(location=[gll.geometry[int(len(gll)/2)].bounds[1], gll.geometry[int(len(gll)/2)].bounds[0]], zoom_start=13)
for i, pt in enumerate(gll.geometry.centroid):
  if os.path.isfile(refFile):
    html = "Area: {:.4f} m2<br> Facade: {:.4f} m2<br> Floors: {:.0f}<br> index: {}<br> Name: {}<br> Address: {} {}<br> Ref Name: {} <br> Ref Address: {} <br> Ref Date: {}".format(gll.loc[i].area, gll.loc[i].facade,
                                                                                                                                                                gll.loc[i].floors, i, gll.loc[i]['name'],
                                                                                                                                                                gll.loc[i]['addr:housenumber'], gll.loc[i]['addr:street'],
                                                                                                                                                                gll.loc[i]['Building Name'], gll.loc[i]['Address'], gll.loc[i]['Date Built'])
  else: html = "Area: {:.4f} m2<br> Facade: {:.4f} m2<br> Floors: {:.0f}<br> index: {}<br> Name: {}<br> Address: {} {}".format(gll.loc[i].area, gll.loc[i].facade, gll.loc[i].floors, i, gll.loc[i]['name'], gll.loc[i]['addr:housenumber'], gll.loc[i]['addr:street'])
           
  iframe = folium.IFrame(html=html, width=200, height=100)
  folium.Marker([pt.y, pt.x], popup=folium.Popup(iframe)).add_to(m)
m

In [ ]:
# Save map to html and data to csv
m.save(name+'.html')
gll['lon'], gll['lat'] = gll['geometry'].centroid.x, gll['geometry'].centroid.y
if os.path.isfile(refFile):
  gll[['lat','lon','name','Building Name','addr:housenumber','addr:street','addr:postcode','Address','Date Built','area','facade','floors']].to_csv(name+'.csv')
else: gll[['lat','lon','name','addr:housenumber','addr:street','addr:postcode','area','facade','floors']].to_csv(name+'.csv')